In [1]:
# import dependencies 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import os
import random

In [2]:
# open browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_mac64.zip
Driver has been saved in cache [/Users/evanbruno/.wdm/drivers/chromedriver/mac64/105.0.5195.52]


In [3]:
# visit nba page 
url = 'https://www.nba.com/stats/teams/boxscores/'
browser.visit(url)

In [4]:
# url used for entering each games data
revise_url = "https://www.nba.com/game/"

In [73]:
# number of pages to click through
for pages in range(1, 4):
    
    #sleep
    time.sleep(random.randint(1, 5))
    
    html = browser.html
    test = soup(html, 'html.parser')
    
    #finds all the hyperlinks on page 
    find = test.find_all("tbody")[2].find_all("a")

    links = []
    # iterates through hyperlinks to find ones linked to game scores
    for link in find:
        if "/stats/game/" in link.get('href'):
            links.append(link.get('href')[12:])
    
    # get dates to name each csv file 
    test_date = []
    names = []
    for date in find:
        if "/games?date" in date.get('href'):
            test_date.append(date.get_text())
            names.append(f"{date.get_text()[:2]}-{date.get_text()[3:5]}")
            
    #iterate through each game link 
    for y in range(len(links)):
        #used to make sure don't click the same game link twice
        if (y % 2) == 0:
            print(y)
            #visit game link 
            browser.visit(f"{revise_url}{links[y]}")
            time.sleep(3)
            #finds box score 
            browser.find_by_id("box-score").click()
            time.sleep(random.randint(1, 5))
            html = browser.html
            test2 = soup(html, 'html.parser')
            
            #obtain column names
            column = []
            for i in range(1, len(test2.find(class_="StatsTable_table__Ejk5X").find_all("th"))):
                column.append(test2.find(class_="StatsTableHead_thead__omZuF").find_all("th")[i].get_text())
            
            #iterate through both teams box scores
            for i in range(2):
                stats = {}
                #get team name
                team_name = test2.find_all(class_="p-4 md:p-5")[i].find(class_="block").get_text()
                
                if i == 0:
                    opponent = test2.find_all(class_="p-4 md:p-5")[1].find(class_="block").get_text()
                else:
                    opponent = test2.find_all(class_="p-4 md:p-5")[0].find(class_="block").get_text()
    
                stats["Opponent"] = []
                #iterate through rows
                for t in range(len(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr"))):
        
                    #iterate through columns to get data 
                    for x in range(1, 21):
                    
                        # comparison used to avoid hitting errors in obtaining wrong data 
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[0].get_text() == "TOTALS":
                            break
                        
                        if x == 1:
                            #adds name of player to dictionary
                            name = (test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find("td").find(class_="GameBoxscoreTablePlayer_gbpNameFull__cf_sn").get_text())
                            stats[name] = []
                        
                        #append nan if player does not play  
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[1].get_text() in ("DNP - Coach's Decision", "DNP - Injury/Illness", "NWT - Personal", "DND - Injury/Illness", "DNP - League Suspension", "DNP - Personal", "DND - Rest", "DND_LEAGUE_SUSPENSION"):
                            stats[name].append(np.nan)
                        
                        #append stats for each player 
                        else:
                            stats[name].append(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[x].get_text())
                            
                # creates dataframe 
                team_data = pd.DataFrame.from_dict(stats, orient='index', columns = column)
                #creates data column
                team_data["Date"] = test_date[y]
                team_data["Opponent"] = opponent
                team_data = team_data.dropna()
                
                #creates float datatype for each column
                for x in ['FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']:
                    team_data[x] = team_data[x].astype(float)
                    
                # calculates players score according to draftkings
                team_data["Score"] = ((team_data["FGM"] - team_data["3PM"]) * 2) * 1 + (team_data["3PM"] * 0.5) + team_data["3PM"] * 3 + team_data["REB"] * 1.25 + team_data["AST"] * 1.5 + team_data["STL"] * 2 + team_data["BLK"] * 2 - team_data["TO"] * 0.5 + team_data["FTM"]
                
                #creates folder for team
                mypath = team_name
                if not os.path.isdir(mypath):
                    os.makedirs(mypath)
                # converts dataframe to csv
                team_data.to_csv(f"{mypath}/{names[y]}.csv", index=True)
    #revisit url
    browser.visit(url)
    
    #click to next page
    for clicker in range(pages):
        browser.find_by_css("button[class='Pagination_button__sqGoH']").click()

0
2
4


KeyboardInterrupt: 

In [74]:
#quit browser
browser.quit()